In [ ]:
!nvidia-smi

Tue Jul  7 23:33:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LSTM
import tensorflow_hub as hub

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Carga de datos

In [ ]:
train_labels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Redes Neuronales/data_taller_2/train_labels.csv')

l_text = []

for i in range(train_labels.shape[0]):
    ID = train_labels['id'][i] 
    train_source = open('/content/drive/My Drive/Colab Notebooks/Redes Neuronales/data_taller_2/train_source_tweets.txt','r')
    for linea in train_source: 
        if str(ID) == linea.split()[0]:
            t = linea.split()[1:]
            text = ''
            for p in t:
                text += p+' '
            l_text.append(text)
            break
    train_source.close()
    
train_labels['text'] = l_text
train_labels

,label,id,text
0,non-rumor,692735698349199360,north korea 'planning some kind of rocket laun...
1,unverified,525008463819464704,"meet kevin vickers, the hero who shot down the..."
2,false,505611045897924608,"15 year old who ""swatted"" gamer convicted of d..."
3,non-rumor,693466724822323200,audio recordings reveal cpr started 11 minutes...
4,true,510922415468449792,awful mt @scclemons:uk aid worker david haines...
...,...,...,...
1921,true,515598200796090369,#workplace hero shoots #altonnolen #stabbings ...
1922,unverified,535257207991205888,russian boy shocked by electricity now claims ...
1923,true,524944399890124801,recap: gunman shot dead inside parliament buil...
1924,false,519965814175567872,a vandal painted a brilliantly simple penis on...


# Encoder

In [ ]:
sent_to_id = {"false":0, "non-rumor":1, "true":2, "unverified":3}
train_labels["label_id"] = train_labels['label'].map(sent_to_id)
train_labels

,label,id,text,label_id
0,non-rumor,692735698349199360,north korea 'planning some kind of rocket laun...,1
1,unverified,525008463819464704,"meet kevin vickers, the hero who shot down the...",3
2,false,505611045897924608,"15 year old who ""swatted"" gamer convicted of d...",0
3,non-rumor,693466724822323200,audio recordings reveal cpr started 11 minutes...,1
4,true,510922415468449792,awful mt @scclemons:uk aid worker david haines...,2
...,...,...,...,...
1921,true,515598200796090369,#workplace hero shoots #altonnolen #stabbings ...,2
1922,unverified,535257207991205888,russian boy shocked by electricity now claims ...,3
1923,true,524944399890124801,recap: gunman shot dead inside parliament buil...,2
1924,false,519965814175567872,a vandal painted a brilliantly simple penis on...,0


In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train_labels.label_id)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)
Y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]])

# Clean text

# Modelo



*   NNLM dim 128 https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2
*   NNLM dim 50 https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2



In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(model, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)

model = Sequential()
model.add(hub_layer)
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_7 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_14 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 68        
Total params: 124,644,820
Trainable params: 124,644,820
Non-trainable params: 0
_________________________________________________________________


# Entrenamiento

In [ ]:
X = train_labels['text']
history = model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.25, verbose=1)

Epoch 1/10
46/46 [==============================] - 2s 35ms/step - loss: 1.3329 - accuracy: 0.3913 - val_loss: 1.2699 - val_accuracy: 0.4523
Epoch 2/10
46/46 [==============================] - 2s 33ms/step - loss: 1.1042 - accuracy: 0.6434 - val_loss: 1.0758 - val_accuracy: 0.5934
Epoch 3/10
46/46 [==============================] - 2s 33ms/step - loss: 0.7607 - accuracy: 0.8442 - val_loss: 0.8288 - val_accuracy: 0.7718
Epoch 4/10
46/46 [==============================] - 2s 33ms/step - loss: 0.4102 - accuracy: 0.9446 - val_loss: 0.6661 - val_accuracy: 0.8050
Epoch 5/10
46/46 [==============================] - 2s 33ms/step - loss: 0.1936 - accuracy: 0.9841 - val_loss: 0.6003 - val_accuracy: 0.8029
Epoch 6/10
46/46 [==============================] - 2s 33ms/step - loss: 0.0931 - accuracy: 0.9965 - val_loss: 0.5708 - val_accuracy: 0.8195
Epoch 7/10
46/46 [==============================] - 2s 33ms/step - loss: 0.0494 - accuracy: 1.0000 - val_loss: 0.5648 - val_accuracy: 0.8257
Epoch 8/10
46

# Predictions

In [ ]:
l_id = []
l_text = []

test_source = open('/content/drive/My Drive/Colab Notebooks/Redes Neuronales/data_taller_2/test_source_tweets.txt','r')
for linea in test_source: 
    ID = linea.split()[0]
    t = linea.split()[1:]
    text = ''
    for p in t:
        text += p+' '
    l_id.append(ID)
    l_text.append(text)

test_source.close()

sample = pd.DataFrame()
sample['id'] = l_id
sample['text'] = l_text

sample["text_cleaned"] = sample["text"].apply(lambda s : clean(s))

Y_pred = np.argmax(model.predict(sample['text_cleaned']), axis=-1)
sample['label_id_pred'] = Y_pred
sent_to_label = {0:"false", 1:"non-rumor", 2:"true", 3:"unverified"}
sample["label_pred"] = sample['label_id_pred'].map(sent_to_label)
sample_sub = pd.DataFrame({'label':sample['label_pred'], 'id':sample['id']})
sample_sub.to_csv('sample_submission_mateneuronas.csv', index=False)
sample_sub.head()

,label,id
0,non-rumor,689719109785907200
1,unverified,531300385248800768
2,false,652992524504600576
3,true,509473920060104704
4,false,651321040119963648
